# Importing libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

# Load the datasets

In [3]:
labels = pd.read_csv("../input/dog-breed-identification/labels.csv")
train_dir = '/kaggle/input/dog-breed-identification/train/'
test_set = '/kaggle/input/dog-breed-identification/test/'
sample = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')
labels.head()

# Preprocessing the data

In [4]:
# Alphabetically ordered labels
dog_breed = sorted(list(set(labels['breed'])))
dog_breed

In [5]:
# Count for categories
categ_count = len(dog_breed)
print(f"We have {categ_count} categories")

In [5]:
# Ascending numeration of alphabetical order
breeds = dict(zip(dog_breed, range(categ_count)))

In [6]:
# Labeling the images
labels['id'] = labels['id'].apply(lambda x: f"{x}.jpg")

# ImageDataGenerator and Data Augmentation

In [7]:
# Create ImageDataGenerator
datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        fill_mode = 'nearest',
        validation_split = 0.2
)

In [7]:
# Setting the default parameters
img_shape = (224, 224, 3)
img_size = (224, 224)
batch_size = 32

In [9]:
# Data Augmentation and splitting into train and validation sets
training_data = datagen.flow_from_dataframe(
    labels,
    directory = '../input/dog-breed-identification/train', 
    x_col = 'id',
    y_col = 'breed',
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training')

valid_data = datagen.flow_from_dataframe(
    labels,
    directory = '../input/dog-breed-identification/train', 
    x_col = 'id',
    y_col = 'breed',
    target_size = img_size,
    batch_size = batch_size,
    class_mode='categorical',
    subset = 'validation')

# Builing the model

* Using InceptionV3 build a pre-trained base model

In [10]:
# load pre-trained InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=img_shape, pooling='avg')

# Freezing/ Unfreezing the layers

for layer in base_model.layers[:-15]:
    layer.trainable = False

for layer in base_model.layers[-15:]:
    layer.trainable = True


In [11]:
# Insert the base_model into the model to fit and compile
x = base_model.output
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(categ_count, activation='softmax')(x)

model = Model(inputs = base_model.input, outputs = predictions)
model.compile(optimizer = Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Fit the model
step_size_train = train_data.n // train_data.batch_size
step_size_valid = val_data.n // val_data.batch_size

history = model.fit_generator(training_data,
                    steps_per_epoch = step_size_train,
                    validation_data = valid_data,
                    validation_steps = step_size_valid,
                    epochs = 50,
                    verbose = 1)

* After 50 iterations model is trained to reach the accuracy of 85%. 

## Plotting the results

In [13]:
# Plot the Accuracy and Loss between Training and Validations sets

plt.figure(figsize=(15,6))

plt.subplot(1,2,1)
plt.plot(history.epoch,history.history['accuracy'],label = 'Training')
plt.plot(history.epoch,history.history['val_accuracy'],label = 'validation')

plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.epoch,history.history['loss'],label = 'Training')
plt.plot(history.epoch,history.history['val_loss'],label = 'validation')

plt.title("Loss")
plt.legend()
plt.show()

## Final model saved

In [14]:
# Save the model
model_name = 'inceptionV3.h5'
model.save(model_name, save_format='h5')

# Checking the model

In [15]:
class_map = training_data.class_indices
classes = []
for key in class_map.keys():
    classes.append(key)

In [29]:
def predict_image(filename, model):
    img_ = image.load_img(filename, target_size=(224, 224))
    img_array = image.img_to_array(img_)
    img_processed = np.expand_dims(img_array, axis=0)
    img_processed /= 255.
    
    prediction = model.predict(img_processed)
    
    index = np.argmax(prediction)
    
    plt.title("Prediction - {}".format(str(classes[index]).title()), size=18, color='red')
    plt.imshow(img_array)

In [30]:
predict_image('../input/testimages/sample2.jpg', model)